<a href="https://colab.research.google.com/github/habstrakT808/How-to-see-spotify-playlist-description-without-limitation/blob/main/Spotify_Playlist_Full_Metadata_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🎵 Spotify Playlist Extractor dengan Spotipy

## 📦 1. Instalasi dan Import Library

In [21]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import os

## 🔐 2. Autentikasi API Spotify


In [22]:
def setup_spotify_client():
    """
    Mengatur koneksi dengan Spotify API menggunakan kredensial
    """
    # Anda bisa mengatur ini sebagai environment variables atau memasukkan langsung
    # Sebaiknya gunakan environment variables untuk keamanan
    client_id = os.getenv('SPOTIFY_CLIENT_ID')
    client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

    if not client_id or not client_secret:
        client_id = input("Masukkan Spotify Client ID: ")
        client_secret = input("Masukkan Spotify Client Secret: ")

    # Mengatur klien Spotify
    client_credentials_manager = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    return sp

## 📂 3. Mendapatkan Informasi Playlist


In [23]:
def get_playlist_info(sp, playlist_url_or_id):
    """
    Mendapatkan informasi detail playlist termasuk deskripsi
    """
    # Ekstrak ID playlist dari URL jika yang dimasukkan adalah URL
    if "spotify.com/playlist/" in playlist_url_or_id:
        playlist_id = playlist_url_or_id.split("spotify.com/playlist/")[1].split("?")[0]
    else:
        playlist_id = playlist_url_or_id

    try:
        # Mendapatkan informasi playlist
        playlist = sp.playlist(playlist_id)

        # Ekstrak informasi utama
        playlist_info = {
            'id': playlist['id'],
            'name': playlist['name'],
            'description': playlist['description'],
            'owner_id': playlist['owner']['id'],
            'owner_name': playlist['owner']['display_name'],
            'followers': playlist['followers']['total'],
            'public': playlist['public'],
            'collaborative': playlist['collaborative'],
            'total_tracks': playlist['tracks']['total'],
            'external_url': playlist['external_urls']['spotify'],
            'image_url': playlist['images'][0]['url'] if playlist['images'] else None
        }

        print(f"✅ Berhasil mengambil informasi playlist: {playlist_info['name']}")
        return playlist_info

    except Exception as e:
        print(f"❌ Error saat mengambil informasi playlist: {e}")
        return None

## 🎶 4. Mendapatkan Semua Lagu dari Playlist

In [24]:
def get_playlist_tracks(sp, playlist_url_or_id, limit=100):
    """
    Mendapatkan semua lagu dari playlist
    """
    # Ekstrak ID playlist dari URL jika yang dimasukkan adalah URL
    if "spotify.com/playlist/" in playlist_url_or_id:
        playlist_id = playlist_url_or_id.split("spotify.com/playlist/")[1].split("?")[0]
    else:
        playlist_id = playlist_url_or_id

    try:
        results = sp.playlist_tracks(playlist_id, limit=limit)
        tracks = results['items']

        # Untuk playlist dengan lagu lebih dari limit, ambil sisa lagu
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])

        # Ekstrak informasi yang dibutuhkan dari setiap lagu
        tracks_info = []
        for i, item in enumerate(tracks):
            track = item['track']

            # Jika track None, lewati (kadang terjadi untuk lagu yang sudah dihapus)
            if not track:
                continue

            # Ekstrak data lagu
            track_data = {
                'track_number': i + 1,
                'track_id': track['id'],
                'track_name': track['name'],
                'artist_name': ', '.join([artist['name'] for artist in track['artists']]),
                'artist_id': ', '.join([artist['id'] for artist in track['artists']]),
                'album_name': track['album']['name'],
                'album_id': track['album']['id'],
                'album_release_date': track['album']['release_date'],
                'duration_ms': track['duration_ms'],
                'popularity': track['popularity'],
                'explicit': track['explicit'],
                'added_at': item['added_at'],
                'added_by': item['added_by']['id'] if item['added_by'] else None,
                'track_url': track['external_urls']['spotify']
            }
            tracks_info.append(track_data)

        print(f"✅ Berhasil mengambil {len(tracks_info)} lagu dari playlist")
        return tracks_info

    except Exception as e:
        print(f"❌ Error saat mengambil lagu dari playlist: {e}")
        return None

## 💾 5. Menyimpan Data ke CSV dan JSON

### 5.1 Simpan ke CSV

In [25]:
def save_to_csv(playlist_info, tracks_info, output_file='playlist_data.csv'):
    """
    Menyimpan informasi lagu ke dalam CSV dengan menambahkan info playlist ke setiap baris
    """
    if not tracks_info:
        print("❌ Tidak ada data lagu untuk disimpan")
        return False

    # Buat dataframe dari informasi lagu
    df = pd.DataFrame(tracks_info)

    # Tambahkan informasi playlist ke setiap baris
    for key, value in playlist_info.items():
        df[f'playlist_{key}'] = value

    # Simpan ke CSV
    df.to_csv(output_file, index=False)
    print(f"✅ Data berhasil disimpan ke {output_file}")

    return True

### 5.2 Simpan ke JSON

In [26]:
def save_playlist_info_json(playlist_info, output_file='playlist_info.json'):
    """
    Menyimpan informasi playlist ke dalam file JSON
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(playlist_info, f, indent=4, ensure_ascii=False)
    print(f"✅ Informasi playlist disimpan ke {output_file}")

## 🚀 6. Fungsi Utama (Main)


In [27]:
def main():
    # Pengaturan koneksi Spotify
    print("=== Pengekstrak Informasi Playlist Spotify ===")
    print("\nMenghubungkan ke Spotify API...")
    sp = setup_spotify_client()

    # Ambil URL atau ID playlist dari user
    playlist_url = input("\nMasukkan URL atau ID playlist Spotify: ")

    # Dapatkan informasi playlist
    print("\nMengambil informasi playlist...")
    playlist_info = get_playlist_info(sp, playlist_url)

    if not playlist_info:
        print("Program berhenti karena tidak bisa mendapatkan informasi playlist.")
        return

    # Tampilkan informasi playlist
    print("\n=== INFORMASI PLAYLIST ===")
    print(f"Nama: {playlist_info['name']}")
    print(f"Deskripsi: {playlist_info['description']}")
    print(f"Pemilik: {playlist_info['owner_name']} ({playlist_info['owner_id']})")
    print(f"Jumlah Lagu: {playlist_info['total_tracks']}")
    print(f"Followers: {playlist_info['followers']}")
    print(f"URL: {playlist_info['external_url']}")

    # Tanya user apakah ingin melanjutkan untuk mengambil data lagu
    choice = input("\nApakah Anda ingin mengambil semua data lagu? (y/n): ")

    if choice.lower() == 'y':
        # Ambil data lagu
        print("\nMengambil data lagu...")
        tracks_info = get_playlist_tracks(sp, playlist_url)

        if tracks_info:
            # Simpan data
            output_csv = input("\nMasukkan nama file untuk menyimpan data CSV (default: playlist_data.csv): ")
            if not output_csv:
                output_csv = "playlist_data.csv"

            save_to_csv(playlist_info, tracks_info, output_csv)

    # Simpan informasi playlist sebagai JSON
    save_json = input("\nApakah Anda ingin menyimpan informasi playlist sebagai JSON? (y/n): ")
    if save_json.lower() == 'y':
        output_json = input("Masukkan nama file JSON (default: playlist_info.json): ")
        if not output_json:
            output_json = "playlist_info.json"
        save_playlist_info_json(playlist_info, output_json)

    print("\nProgram selesai!")

## Jalankan Program

In [28]:
if __name__ == "__main__":
    main()

=== Pengekstrak Informasi Playlist Spotify ===

Menghubungkan ke Spotify API...
Masukkan Spotify Client ID: 587a34de10c845f4aeb2a29e96edf734
Masukkan Spotify Client Secret: 9b03217cbd4140d084190098ae66f021

Masukkan URL atau ID playlist Spotify: https://open.spotify.com/playlist/3wlLPheGvVHz0t2GsyAQRe?si=c34694ee73f34178

Mengambil informasi playlist...
✅ Berhasil mengambil informasi playlist: A Letter I've Never Sent #2

=== INFORMASI PLAYLIST ===
Nama: A Letter I've Never Sent #2
Deskripsi: But if u ever stumble upon this, just know.... I never forgot. Not u, not us, not the way u made me feel. I dont expect anything, not even a reply. I just needed to say this somewhere, somehow. Sooo.. this became A Letter I&#x27;ve Never Sent, the only way i knew how to say what i felt.
Pemilik: Raluv A. (31orvnbq2efbgtlha7anut2ueg4i)
Jumlah Lagu: 41
Followers: 2
URL: https://open.spotify.com/playlist/3wlLPheGvVHz0t2GsyAQRe

Apakah Anda ingin mengambil semua data lagu? (y/n): y

Mengambil data lag